# Set up

### Imports

For the feature extraction process we need the file of file-infromation from the previous step. To use a pre-ran file follow the step below. 


To run this code you need a google account, and to follow the link below and create a shortcut to the folder and store it in MyDrive:
https://drive.google.com/drive/folders/1YHmpr-a8FPFA3dOKW2fj4No_QOiY9AjM

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import librosa
import numpy as np
import pandas as pd 
import os 

from scipy import signal
from scipy.stats import norm, kurtosis
from scipy.signal import welch
from scipy.integrate import simps

import operator
import glob 
import json
from datetime import datetime

from tqdm import tqdm

Path to where the shortcut folder is stored, if it is stored to MyDrive the following code should take us there

In [4]:
%cd gdrive/MyDrive/Experiments-feature-files

/content/gdrive/.shortcut-targets-by-id/1YHmpr-a8FPFA3dOKW2fj4No_QOiY9AjM/Experiments-feature-files


Loading in the data

In [6]:
#Importing table of file information from step 0
df_paths = pd.read_csv("wav_paths-demo.csv") 
#df_paths = pd.read_csv("/content/gdrive/MyDrive/Masteroppgave/wav_paths_master.csv") 

df_paths['mic'] = df_paths['mic'].str.replace("l", "").str.replace("m", "").apply(pd.to_numeric) #Removes 'm' and 'l' from mic and casts it to int 

In [8]:
df_paths

,path,leak_type,environment,recording,mic,knob_rotations,leak_present
0,/content/gdrive/MyDrive/IDMT_dataset/ventleak/...,ventleak,hydr,1,3,6.5n,1
1,/content/gdrive/MyDrive/IDMT_dataset/ventleak/...,ventleak,hydr,1,3,8.5n,1
2,/content/gdrive/MyDrive/IDMT_dataset/ventleak/...,ventleak,hydr,1,4,5.0n,0
3,/content/gdrive/MyDrive/IDMT_dataset/ventleak/...,ventleak,hydr,1,4,4.5n,0
4,/content/gdrive/MyDrive/IDMT_dataset/ventleak/...,ventleak,hydr,1,4,7.5n,1
...,...,...,...,...,...,...,...
5587,/content/gdrive/MyDrive/IDMT_dataset/tubeleak/...,tubeleak,hydr,3,3,3.5n,0
5588,/content/gdrive/MyDrive/IDMT_dataset/tubeleak/...,tubeleak,hydr,3,2,1.0n,0
5589,/content/gdrive/MyDrive/IDMT_dataset/tubeleak/...,tubeleak,hydr,3,4,7.0n,1
5590,/content/gdrive/MyDrive/IDMT_dataset/tubeleak/...,tubeleak,hydr,3,4,8.0n,1


# Functions

## Helper functions 

In [9]:
def join_meta(df_meta, df_input):
  df = df_input.join(df_meta)
  df.index.name = "index"
  return df

In [10]:
def pre_save_files(df_input, cutoff=0, save_path=None, sr=None):
  df_files = pd.DataFrame()
  
  for ind in tqdm(df_input.index):
    y, sample_rate = librosa.load(df_input['path'][ind], sr=None)
    if type(cutoff) == int and cutoff > 0: 
      y = butter_highpass_filter(y, cutoff, sample_rate,order=4)
    df_file = pd.DataFrame({"samples": [y], "sr": sample_rate})
    df_files = pd.concat([df_files, df_file], join = 'outer', axis = 0)
  df_files.index = df_input.index
  
  if save_path:
    df_files.to_json(save_path)


  return df_files

In [11]:
def preprocesses(X,T,normalize=True):
  if normalize:
    scaler = MinMaxScaler()
    scaler = scaler.fit(X)
    X_scaled = pd.DataFrame(scaler.transform(X))
    X_scaled.columns = X.columns
    T_scaled = pd.DataFrame(scaler.transform(T))
    T_scaled.columns = T.columns

  return X_scaled, T_scaled 


In [12]:
def make_list(split_list): #[0,10000,1000]
  split = split_list.copy()
  split[1] = split[1]+split[-1]
  splitted_list =  np.arange(split[0], split[1], split[2]).tolist()
  return splitted_list

In [13]:
def get_wanted_files(df, wanted_files):
    for key in wanted_files:
        if wanted_files[key]:
            df = df.loc[(df[key].isin(wanted_files[key]))]
    return df 


In [14]:
def bandpower(data, sf, band_list, window_sec=None, relative=False, reduce_split=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.
    """


    # Define window length
    if window_sec is not None:
        nperseg = window_sec * sf
    else:
        nperseg = 0.3 * sf

    # Compute the modified periodogram (Welch)
    #win = scipy.signal.get_window('hanning',nperseg)
    freqs, psd = welch(data, sf, nperseg=nperseg)
    
    avg_band_power = []

    #Kan vurdere å bruke matplotlib sin psd 
    ##win = matplotlib.mlab.window_hanning(np.ones(nperseg)) # or
    ##psd, freqs = matplotlib.mlab.psd(data, Fs=sf,window=win)



    # Frequency resolution
    for i in range(len(band_list)-1):
      freq_res = freqs[1] - freqs[0]
      low = band_list[i]
      high = band_list[i+1]

      # Find closest indices of band in frequency vector
      idx_band = np.logical_and(freqs >= low, freqs <= high)

      # Integral approximation of the spectrum using Simpson's rule.
      bp = simps(psd[idx_band], dx=freq_res)


      if relative:
          bp /= simps(psd, dx=freq_res)

      
      avg_band_power.append(bp)


    if reduce_split:
      reduced = []  
      for i in range(len(reduce_split)-1):
        s = avg_band_power[reduce_split[i]:reduce_split[i+1]]
        reduced.append(sum(s))
      return pd.DataFrame([reduced])

    return pd.DataFrame([avg_band_power])

## Feature Extraction

In [15]:
def get_train_test_dataframes(df, wanted_files):    
    df_wanted_files = get_wanted_files(df, wanted_files)

    df_wanted_files_loaded = pre_save_files(df_wanted_files)

    df_wanted_files = join_meta(df_wanted_files_loaded, df_wanted_files)
        
    return df_wanted_files



In [16]:
def get_train_test_features(df_wanted, use_features, split_list, relative_power, win_len, log_rel_pwr, shuffle):

  print("> making features")
  df_wanted_features = get_all_features(df_wanted, use_features, split_list, relative_power, win_len,log_rel_pwr=False) 
  df_wanted_features = df_wanted_features.loc[:, ~df_wanted_features.columns.duplicated()]
  df_wanted_features = df_wanted_features.join(df_wanted, on="feature_index", how="left")
  print("")

  if log_rel_pwr:
    x_norm = df_wanted_features.filter(regex='rp_bin')
    x_norm_log = np.log(x_norm)
    df_wanted_features.loc[x_norm_log.index.isin(df_wanted_features.index), x_norm_log.columns.values] = x_norm_log[x_norm_log.columns.values]


  return df_wanted_features


In [17]:
#GETTING FREQ. AND REL-PWR FEATURES AND COMBINING THEM
def get_all_features(df, use_features, split_list, relative_power, win_len=0.3, log_rel_pwr=False):

  if "time-freq" in use_features: 
    print("time-frequency features")
    df_freq = get_time_freq_feature_dataframe(df, win_len=win_len)
  else: 
    df_freq = pd.DataFrame()

  if "rel-pwr" in use_features: 
    print("relative power features")
    df_relative_pwr = get_bandpower_dataframe_framed(df,win_len=win_len, window_sec=1, split_list=split_list, relative_power=relative_power)
    print("done!")
  else: 
    df_relative_pwr = pd.DataFrame()


  df_freq.reset_index(inplace=True)
  df_relative_pwr.reset_index(inplace=True)
  
  df_all_features = pd.concat([df_freq,df_relative_pwr],axis=1)

  df_all_features.columns = df_all_features.columns.astype(str)

  return df_all_features 
  

In [33]:
def get_time_freq_feature_dataframe(df_input, win_len):
  FRAME_LENGTH = int((win_len * 48000)) #48000 is the expected sample rate
  HOP_LENGTH = FRAME_LENGTH // 2 #50% overlap

  features = []

  for ind in tqdm(df_input.index):
    feature_list_freq = freq_domain_features(df_input["samples"][ind],df_input["sr"][ind], ind, FRAME_LENGTH=FRAME_LENGTH,HOP_LENGTH=HOP_LENGTH)

    #feature_list = pd.concat([feature_list_time, feature_list_freq], join = 'outer', axis = 1)
    
    features.append(feature_list_freq)

  df = pd.concat(features, ignore_index=True)


  return df


In [19]:
#FREQUNECY DOMAIN FEATURES 
def freq_domain_features(signal, sr, index, FRAME_LENGTH,HOP_LENGTH, no_panda=False,):

  frames = librosa.util.frame(signal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH).T #Overlappen skal komme fra hop_length her som er 50%

  #RETURNS 2D FEATURE LIST 
  spec_contrast = (librosa.feature.spectral_contrast(y=signal, sr=sr, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, center=False))#.mean(axis=0)  
  df_contrast = pd.DataFrame(spec_contrast).T
  names_contrast = [str("cont_bin: " + str(i)) for i in range(len(np.array(df_contrast.columns)))]
  df_contrast.columns = names_contrast
  
  mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=6, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, center=False) #Using 6 MFCC bins 
  df_mfcc = pd.DataFrame(mfcc).T
  names_mfcc = [str("mfcc_bin: " + str(i)) for i in range(len(np.array(df_mfcc.columns)))]
  df_mfcc.columns = names_mfcc
  
  #RETURNS 1D FEAUTRE LIST 
  zcr = librosa.feature.zero_crossing_rate(y=signal,frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH,center=False)[0]
  rmse=librosa.feature.rms(y=signal,frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH,center=False)[0]
  spec_centroid = (librosa.feature.spectral_centroid(y=signal, sr=sr, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH,center=False)[0])
  spec_bandwidth = (librosa.feature.spectral_bandwidth(y=signal, sr=sr, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH,center=False)[0])
  spec_flatness = (librosa.feature.spectral_flatness(y=signal, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH,center=False)[0])
  spec_rolloff = (librosa.feature.spectral_rolloff(y=signal, sr=sr, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH,center=False)[0])

  flat_features = [spec_centroid, spec_bandwidth, spec_flatness, zcr, rmse] 
  flat_names = ["spec_centroid", "spec_bandwidth", "spec_flatness", "zcr", "rmse"]
  df_flat = pd.DataFrame(flat_features).T
  df_flat.columns = flat_names 

  df = pd.concat([df_flat,df_contrast,df_mfcc],axis=1)
  index_col = (np.ones(len(frames))*index)
  #df["feature_index"] = index_col
  df.insert(0, "feature_index", index_col)

  return df

In [20]:
#RELATIVE-POWER FEATURES
def get_bandpower_dataframe_framed(df_input, win_len, window_sec, split_list, relative_power, reduce_split=False):
  df = pd.DataFrame()

  if split_list[0] == "manual": 
    band_list = split_list[1]
  else: 
    band_list = make_list(split_list[1])

  #for ind in tqdm(df_input.index):
  for ind in (df_input.index):

    df_signal = pd.DataFrame()

    signal, sr = np.array(df_input["samples"][ind]), df_input["sr"][ind]

    FRAME_LENGTH = int(sr * win_len)
    HOP_LENGTH = FRAME_LENGTH//2
    frames = librosa.util.frame(signal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH).T 
    
    for frame in frames:
      bin_power = bandpower(frame, sr, band_list ,window_sec=window_sec, relative=relative_power, reduce_split=reduce_split)
      bin_power['index'] = [float(ind)]
      
      df_signal = pd.concat([df_signal,bin_power],axis=0)
    
    df = pd.concat([df,df_signal],axis=0,)

  names = [str("rp_bin: " + str(i)) for i in range(len(np.array(df.columns)))]
  df.columns = names
  df.columns = [*df.columns[:-1], 'feature_index']

  return df



# Extracting and saving features

In [36]:
quick_test = {
    'leak_type' : ["ventleak"],
    'environment' : ["lab"],
    'recording' : [2],
    'mic' : [1]
}

In [21]:
#EXPERIMENTS FOR TREE-BASED GAS LEAK DETECTION
experiment_files_E1toE3 = {
    'leak_type' : ["ventleak", "tubeleak", "ventlow"],
    'environment' : ["work", "work_low", "hydr", "hydr_low", "lab"],
    'mic' : [1]
}

experiment_files_E4 = {
    'leak_type' : ["ventleak", "tubeleak"],
    #'leak_type' : ['ventleak'],
    'environment' : ["work", "hydr"],
    'mic' : [1,2,3]
}

experiment_features_E1toE4 = {
    'win_len' : 5, 
    'log_rel_pwr' : True, 
    'normalize' : False,
    'shuffle' : False,
    'split_list' : ["manual",[0,10000,20000,24000]],
    'relative_power' : False,
    'use_features' : [
                      'time-freq',
                      'rel-pwr'
                      ],      
}

In [22]:
#EXPERIMENTS FOR OCC ANOMALY DETECTION
experiment_files_OCC = {
    'leak_type' : ["ventleak", "tubeleak", "ventlow"],
    'environment' : ["work","hydr", "work_low", "hydr_low","lab"],
    'mic' : [4]
}

experiment_features_OCC = {
    'win_len' : 5, 
    'log_rel_pwr' : True, 
    'normalize' : False,
    'shuffle' : False,
    'split_list' : ["even", [0,24000,4000]],
    'relative_power' : False,
    'use_features' : [
                      'time-freq',
                      'rel-pwr'
                      ],          
}

In [37]:
#Method that uses the information from the dictionaries to use the relevant files and the specified features 
def extract_features(df_paths, experiment_files, experiment_features, save_path=None):
  df_wanted_files = get_train_test_dataframes(df_paths, experiment_files)
  df_wanted_files_features = get_train_test_features(df_wanted_files, use_features=experiment_features["use_features"], split_list=experiment_features["split_list"], relative_power=experiment_features["split_list"], win_len=experiment_features["win_len"], log_rel_pwr=experiment_features["log_rel_pwr"], shuffle=experiment_features["shuffle"])
  if save_path: 
    df_wanted_files_features.to_csv(save_path)


Below are the extract_features method executed to get features for all experiments, as well as a quick test run

In [39]:
extract_features(df_paths, quick_test, experiment_features=experiment_features_E1toE4, save_path="quick_test.csv")

100%|██████████| 32/32 [00:00<00:00, 59.73it/s]


> making features
time-frequency features


100%|██████████| 32/32 [00:39<00:00,  1.23s/it]


relative power features
done!



In [ ]:
#E1, E2 and E3
extract_features(df_paths, experiment_files_E1toE3, experiment_features=experiment_features_E1toE4, save_path="/content/gdrive/MyDrive/Masteroppgave/extracted_features/Gas_Leak_E1-E2-E3_train_test_features.csv")

In [ ]:
#E4
extract_features(df_paths, experiment_files_E4, experiment_features=experiment_features_E1toE4, save_path="/content/gdrive/MyDrive/Masteroppgave/extracted_features/Gas_Leak_E4_train_test_features.csv")

In [ ]:
#OCC
extract_features(df_paths, experiment_files_OCC, experiment_features=experiment_features_OCC, save_path="/content/gdrive/MyDrive/Masteroppgave/extracted_features/Anomaly_Detection_train_test_features.csv")